In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [7]:
data=pd.read_excel(r'C:\Users\utkar\OneDrive\Desktop\EDA_project\Data_set\Split_data_final.xlsx', sheet_name ='Sheet1') # Reading data 

In [8]:
data_A=pd.DataFrame()

In [9]:
data_L=pd.DataFrame()

In [10]:
data_N=pd.DataFrame()

In [11]:
data_A=data.loc[data['AV'] == 'A']

In [12]:
data_L=data.loc[data['AV'] == 'L']

In [13]:
data_N=data.loc[data['AV'] == 'N']

In [14]:
data['Threat'] = data['Threat'].str.lower()
data['Threat'] = data['Threat'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data["Threat"] = data['Threat'].str.replace('[^\w\s]','')

C:\Users\utkar\AppData\Local\Temp\ipykernel_24936\1371494755.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data["Threat"] = data['Threat'].str.replace('[^\w\s]','')


import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = data_A.Threat.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

In [15]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
new_stopwords = ["id", "from", "could", "use", "also","Many","like","May","may","ca","even","ect","i.e","cause","result","used","form","Its","cause","take","following"]
stop.extend(new_stopwords)

data['Threat']=data['Threat'].apply(lambda x: [item for item in x if item not in stop])

In [16]:
from nltk.tokenize import sent_tokenize, word_tokenize

data_A['Threat']=data_A['Threat'].apply(word_tokenize)

C:\Users\utkar\AppData\Local\Temp\ipykernel_24936\902507766.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_A['Threat']=data_A['Threat'].apply(word_tokenize)


In [17]:
import nltk, re, string, collections
from nltk.util import ngrams
data_lemmatized=data_A['Threat-bigrams'] = data_A['Threat'].apply(lambda row: list(nltk.bigrams(row)))
data_A['Threat-bigrams'].head()

C:\Users\utkar\AppData\Local\Temp\ipykernel_24936\2303327044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lemmatized=data_A['Threat-bigrams'] = data_A['Threat'].apply(lambda row: list(nltk.bigrams(row)))


8      [(It, was), (was, discovered), (discovered, th...
50     [(It, was), (was, discovered), (discovered, th...
79     [(It, was), (was, discovered), (discovered, th...
101    [(It, was), (was, discovered), (discovered, th...
216    [(The, kernel), (kernel, packages), (packages,...
Name: Threat-bigrams, dtype: object

import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_A['Threat-bigrams'])

# Create Corpus
texts = data_A['Threat-bigrams']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

In [ ]:
!pip install spacy

In [19]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = data_A.Threat.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['it', 'was', 'discovered', 'that', 'sane', 'backends', 'incorrectly', 'handled', 'certain', 'packets', 'it', 'was', 'discovered', 'that', 'sane', 'backends', 'incorrectly', 'handled', 'certain', 'memory', 'operations']


In [20]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [21]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [22]:
data_words_bigrams = make_bigrams(data_words)

import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

import spacy
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

In [23]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 2)]


In [24]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [25]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"linux" + 0.007*"red" + 0.005*"the" + 0.005*"hat" + 0.005*"kernel" + '
  '0.004*"enterprise" + 0.004*"for" + 0.004*"in" + 0.003*"discovered" + '
  '0.003*"it"'),
 (1,
  '0.065*"kernel" + 0.057*"linux" + 0.050*"cve" + 0.045*"for" + 0.044*"red" + '
  '0.044*"hat" + 0.033*"in" + 0.025*"enterprise" + 0.022*"to" + '
  '0.020*"overflow"'),
 (2,
  '0.092*"linux" + 0.086*"hat" + 0.086*"red" + 0.081*"enterprise" + '
  '0.056*"for" + 0.026*"kernel" + 0.025*"power" + 0.025*"ppc" + 0.025*"update" '
  '+ 0.024*"server"'),
 (3,
  '0.079*"the" + 0.037*"it" + 0.036*"that" + 0.034*"discovered" + 0.034*"was" '
  '+ 0.029*"kernel" + 0.026*"in" + 0.026*"linux" + 0.019*"properly" + '
  '0.019*"not"'),
 (4,
  '0.086*"the" + 0.068*"in" + 0.048*"was" + 0.045*"discovered" + 0.045*"it" + '
  '0.045*"that" + 0.044*"kernel" + 0.044*"linux" + 0.032*"properly" + '
  '0.032*"not"'),
 (5,
  '0.065*"vmware" + 0.052*"esxi" + 0.052*"kb" + 0.045*"com" + 0.045*"http" + '
  '0.042*"externalid" + 0.042*"selfse

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [27]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [32]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('C:\Users\utkar\OneDrive\Desktop\EDA_project\Data_set.csv', index=False)
    pbar.close()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (4106075069.py, line 59)

In [33]:
print(model_results)

{'Validation_Set': ['75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', '75% Corpus', 